In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
from sys import exit
from matplotlib.backends.backend_pdf import PdfPages 
import matplotlib.dates as mdates
import seaborn as sns
from calendar import monthrange
import datetime
from tqdm import tqdm
import ast
import os
from scipy import stats
from scipy.optimize import curve_fit
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# This is a script copy of "Script11.py", ported to JupyterNotebook for ease.

In [3]:
# COLDEST DAY ANALYSIS, Hour in Index, EPC in Columns - drop nan

In [4]:
def GetPropertyDetails(DataRow):
    EPCRating = DataRow['House_SAP'].values[0]
    FloorArea = DataRow['Total_Floor_Area'].values[0]
    Employment = DataRow['EmployType'].values[0]
    HouseIncome = DataRow['HouseIncome'].values[0]
    Tenure = DataRow['Tenure'].values[0]
    HPType = DataRow['HP_Installed'].values[0]
    HP_Size_kW = DataRow['HP_Size_kW'].values[0]
    Age = DataRow['Participant_Age'].values[0]
    JobStatus = DataRow['EmployStatus'].values[0]
    Fluid = DataRow['HP_Refrigerant'].values[0]
    MCS_SHLoad = DataRow['MCS_SHLoad'].values[0]
    MCS_Hloss = DataRow['MCS_Hloss'].values[0]
    return(EPCRating,FloorArea,Employment,HouseIncome,Tenure,HPType,HP_Size_kW,Age,JobStatus,Fluid,MCS_SHLoad,MCS_Hloss)
#
def PlotLimits(Label):
    if Label == 'Hourly_Heat_Pump_Power_Consumed':
        start,end = 0,5
    elif Label == 'Internal_Air_Temperature':
        start,end = 10,30
    elif Label == 'External_Air_Temperature':
        start,end = -15,40          
    elif Label == 'Temperature_Makeup':
        start,end = -5,35            
    elif Label == 'COP_H4':
        start,end = 0,5
    else:
        start,end = 10,30
    return(start,end)
#
def OccuranceAndInstances(Hue,AllList):
    TempList = [Hue]
    UniqueList = set(AllList)
    for items in UniqueList:
        Occurence = AllList.count(items)
        Vals = f"{items} occurs {round(Occurence/24,1)} times ## "
        TempList.append(Vals)
    return(TempList)
#
def GetPolyCoefficients(XValues,YValues):
    coeff_A1,YIntercpt_A1 = np.polyfit(XValues,YValues,deg=1) # linear plot
    LinearFit = coeff_A1*X + YIntercpt_A1
    coeff_B1,coeff_B2,YIntercptB = np.polyfit(XValues,YValues,deg=2) # polynominal plot.
    PolyFit = coeff_B1*X*X + coeff_B2*X + YIntercptB
    return(LinearFit,coeff_A1,YIntercpt_A1,PolyFit,coeff_B1,coeff_B2,YIntercptB)

In [5]:
LogFile = open("3.Methodology/HPDemonstrationProject/LogFile_ColdestDay.Log",mode="w")

In [6]:
DataTable = pd.DataFrame(columns=['Property_ID','Year','Month','Day','Hour','Hourly_Heat_Pump_Power_Consumed','Internal_Air_Temperature','External_Air_Temperature',
                                  'Temperature_Makeup','COP_H4','EPCRating','Employment','HouseIncome','Tenure','HPType','HP_Size_kW','Age','JobStatus','HP_Refrigerant','MCS_Characteristics'])

PropDesignTbl = pd.read_csv("../Data/UK Data Service/Electrification of Heat Demonstration Project 2020-2022/BEIS Electrification of Heat Project - Property, Design and Installation Information.csv")
SummaryTable = pd.read_csv("../Data/UK Data Service/Electrification of Heat Demonstration Project 2020-2022/BEIS Electrification of Heat Project - Interim Heat Pump Performance Data Summary.csv")
MasterDF = pd.read_csv("../Data/UK Data Service/Electrification of Heat Demonstration Project 2020-2022/Resampled_60min/000MasterTable_60min.csv",index_col='Unnamed: 0')
Summ_ColdestDay = SummaryTable.loc[(SummaryTable['Included_SPF_analysis'] == True) & (SummaryTable['Included_coldest_day_analysis'] == True)]
Summ_ColdestDay['Coldest_day_start']= pd.to_datetime(Summ_ColdestDay['Coldest_day_start'])
ValidPropertyIDs = Summ_ColdestDay['Property_ID'].tolist()
ColdestDay = Summ_ColdestDay['Coldest_day_start'].tolist()
#
#
print("Reading Complete, now extracting ...")
for propID, dates in zip(ValidPropertyIDs,ColdestDay):
    PropertyLine = PropDesignTbl.loc[(PropDesignTbl['Property_ID'] == propID)].copy()
    EPCRating,FloorArea,Employment,HouseIncome,Tenure,HPType,HP_Size_kW,Age,JobStatus,Fluid,MCS_SHLoad,MCS_Hloss = GetPropertyDetails(PropertyLine)
    #
    Year,Month,Day = dates.year, dates.month, dates.day
    ColdestDay = MasterDF.loc[(MasterDF['Property_ID'] == propID) & (MasterDF['Year'] == Year) & (MasterDF['Month'] == Month) & (MasterDF['Day'] == Day)].copy()
    for Hour in range(0,24):
        DataList = []
        #
        DataHour = ColdestDay.loc[(ColdestDay['Hour'] == Hour)]
        HPPower,InterAirTemp,ExtAirTemp,TempMakeUp,COP_H4 = float('nan'),float('nan'),float('nan'),float('nan'),float('nan')
        if (DataHour['Hourly_Heat_Pump_Power_Consumed'].any()) == True:
            HPPower = round(DataHour['Hourly_Heat_Pump_Power_Consumed'].values[0],3)
            InterAirTemp = round(DataHour['Internal_Air_Temperature'].values[0],3)
            ExtAirTemp = round(DataHour['External_Air_Temperature'].values[0],3)
            TempMakeUp = round(DataHour['Temperature_Makeup'].values[0],3)
            COP_H4 = round(DataHour['COP_H4'].values[0],3)
            MCS_Character = DataHour['MCS_Characteristics'].values[0]
        DataList.extend([propID,Year,Month,Day,Hour,HPPower,InterAirTemp,ExtAirTemp,TempMakeUp,COP_H4,EPCRating,Employment,HouseIncome,Tenure,HPType,HP_Size_kW,Age,JobStatus,Fluid,MCS_Character])
        DataTable.loc[len(DataTable), DataTable.columns] = (DataList)
DataTable.to_csv("3.Methodology/HPDemonstrationProject/DataTable.csv") 

Reading Complete, now extracting ...


In [7]:
print("########### SEASONAL DATA ###########",file=LogFile)
Seasons = ['Winter','Summer','Spring','Autumn']
Parameters = ['Hourly_Heat_Pump_Power_Consumed','Internal_Air_Temperature','External_Air_Temperature','Temperature_Makeup','COP_H4']
HueVarients = ['EPCRating','MCS_Characteristics','JobStatus','Employment','HouseIncome','Tenure','HPType','HP_Size_kW','HP_Refrigerant']
for Season in Seasons:
    plt.figure(figsize=(100,30))
    fignums_1 = 1
    SasonalDF = MasterDF.loc[(MasterDF['Seasons'] == Season)]
    for Param in Parameters:
        for Hue in tqdm(HueVarients,desc=f"{Season} ## {Param} ## Plotting ..."):
            if Param == Parameters[0]: # Only on the first one - rest will just be a repeat.
                data = SasonalDF[Hue].tolist()
                returndata = OccuranceAndInstances(Hue,data)
                print(f"{Season} ## {Param} ## {returndata}",file=LogFile)
            StartLim,EndLim = PlotLimits(Param)
            PvtTable = pd.pivot_table(SasonalDF, values=Param, index=['Hour'],columns=Hue,aggfunc='mean')
            plt.subplot(len(Parameters),len(HueVarients),fignums_1)
            plt.title(f'{Season} ## Fig no:{fignums_1} = #{Param}# with varients of #{Hue}#.')
            sns.lineplot(PvtTable,dashes=False,linewidth=1,markers=True)
            plt.ylabel(Param)
            plt.ylim(StartLim,EndLim)
            plt.xticks(range(0,24,2))
            plt.grid(which='both',linewidth = 0.5)
            plt.tight_layout() 
            fignums_1 += 1
    plt.savefig(f"3.Methodology/HPDemonstrationProject/1_{Season}_SesonalAnalysis.png")
#MasterDF = 0    # Releasing memory usage.
#SasonalDF = 0   # Releasing memory usage.
plt.close('all')

Winter ## Hourly_Heat_Pump_Power_Consumed ## Plotting ...: 100%|██████████| 9/9 [00:03<00:00,  2.43it/s]
Winter ## Internal_Air_Temperature ## Plotting ...: 100%|██████████| 9/9 [00:05<00:00,  1.72it/s]
Winter ## External_Air_Temperature ## Plotting ...: 100%|██████████| 9/9 [00:07<00:00,  1.18it/s]
Winter ## COP_H4 ## Plotting ...: 100%|██████████| 9/9 [00:12<00:00,  1.38s/it]
Summer ## Hourly_Heat_Pump_Power_Consumed ## Plotting ...: 100%|██████████| 9/9 [00:02<00:00,  3.48it/s]
Summer ## Internal_Air_Temperature ## Plotting ...: 100%|██████████| 9/9 [00:04<00:00,  2.06it/s]
Summer ## External_Air_Temperature ## Plotting ...: 100%|██████████| 9/9 [00:07<00:00,  1.23it/s]
Summer ## COP_H4 ## Plotting ...: 100%|██████████| 9/9 [00:11<00:00,  1.25s/it]
Spring ## Hourly_Heat_Pump_Power_Consumed ## Plotting ...: 100%|██████████| 9/9 [00:03<00:00,  2.54it/s]
Spring ## Internal_Air_Temperature ## Plotting ...: 100%|██████████| 9/9 [00:05<00:00,  1.69it/s]
Spring ## External_Air_Temperature 

In [21]:
MasterDF = pd.read_csv("../Data/UK Data Service/Electrification of Heat Demonstration Project 2020-2022/Resampled_60min/000MasterTable_60min.csv",index_col='Unnamed: 0')
SasonalDF = MasterDF.loc[(MasterDF['Seasons'] == 'Winter')]
PvtTable = pd.pivot_table(SasonalDF, values='COP_H4', index=['Hour'],columns='EPCRating',aggfunc='mean')
PvtTable.mean()

EPCRating
A    3.297817
B    2.898577
C    2.989854
D    3.030289
E    2.811295
F    3.165598
dtype: float64

In [8]:
print("\n\n########### COLDEST DAY ###########",file=LogFile)
DataTable = pd.read_csv("3.Methodology/HPDemonstrationProject/DataTable.csv",index_col='Unnamed: 0')
plt.figure(figsize=(100,30))
fignums_2 = 1
for Param in Parameters:
    for Hue in tqdm(HueVarients,desc=f"Coldest Day ## {Param} ## Plotting ..."):
        if fignums_2 < len(HueVarients):
            data = DataTable[Hue].tolist()
            returndata = OccuranceAndInstances(Hue,data)
            print(returndata,file=LogFile)
        #
        StartLim,EndLim = PlotLimits(Param)
        PvtTable = pd.pivot_table(DataTable, values=Param, index=['Hour'],columns=Hue,aggfunc='mean')
        plt.subplot(len(Parameters),len(HueVarients),fignums_2)
        plt.title(f'Fig no:{fignums_2} = #{Param}# with varients of #{Hue}#.')
        sns.lineplot(PvtTable,dashes=False,linewidth=1,markers=True)
        plt.ylabel(Param)
        plt.ylim(StartLim,EndLim)
        plt.xticks(range(0,24,2))
        plt.grid(which='both',linewidth = 0.5)
        plt.tight_layout() 
        fignums_2 += 1
plt.savefig("3.Methodology/HPDemonstrationProject/2_ColdestDayAnalysis.png")
plt.close('all')

Coldest Day ## Hourly_Heat_Pump_Power_Consumed ## Plotting ...: 100%|██████████| 9/9 [00:02<00:00,  3.32it/s]
Coldest Day ## Internal_Air_Temperature ## Plotting ...: 100%|██████████| 9/9 [00:05<00:00,  1.52it/s]
Coldest Day ## External_Air_Temperature ## Plotting ...: 100%|██████████| 9/9 [00:08<00:00,  1.03it/s]
Coldest Day ## Temperature_Makeup ## Plotting ...: 100%|██████████| 9/9 [00:11<00:00,  1.32s/it]
Coldest Day ## COP_H4 ## Plotting ...: 100%|██████████| 9/9 [00:13<00:00,  1.54s/it]


In [9]:
X = np.arange(0,40)
ASHPCOP = 4.08-0.09*X+0.0008*X*X
#
plt.figure(figsize=(16,9))
DataTable.dropna(subset = ['Temperature_Makeup', 'COP_H4'],inplace=True)
XValues = DataTable["Temperature_Makeup"]
YValues = DataTable["COP_H4"]
plt.title('Coldest day COP performance.')
sns.scatterplot(DataTable,x="Temperature_Makeup",y="COP_H4",size=0.05,legend=False,linewidth=0)
sns.lineplot(x=X, y=ASHPCOP, color = 'red',label='Initial Guess = 4.08-0.09*X+0.0008*X*X',linewidth=5)
LinearFit,coeff_A1,YIntercpt_A1,PolyFit,coeff_B1,coeff_B2,YIntercptB = GetPolyCoefficients(XValues,YValues)
sns.lineplot(x=X,y=LinearFit, color='black',label=f'Linear = ({round(coeff_A1,4)}*X + {round(YIntercpt_A1,4)})',linewidth=5)
sns.lineplot(x=X,y=PolyFit, color='purple',label=f'Poly = ({round(coeff_B1,4)}*X*X + {round(coeff_B2,4)}*X + {round(YIntercptB,3)})',linewidth=5)
plt.grid(which='both',linewidth = 0.5)
plt.tight_layout() 
plt.savefig("3.Methodology/HPDemonstrationProject/2_ColdestDayAnalysis_COP.png")
plt.close('all')
#
LogFile.close()
DataTable = 0    # Releasing memory usage.